<a href="https://colab.research.google.com/github/mushrafi88/solar_irradiation_forecasting/blob/main/keras_one_quantum_node_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Libraries

In [ ]:
!pip install pip install tensorflow==2.7.0
!pip install pennylane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.7/489.7 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.1/463.1 KB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.5 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.11.0
    Uninstalling tensorflow-estimator-2.11.0:
      Successfully uninstalled tensorflow-estimator-2.11.0
  Attempting uninstall: keras
    Found existing installation: keras 2.11.0
    Uninstalling keras-2.11.0:
      Successfully uninstalled keras-2.11.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 23.3.3
    Uninstalling flatbuffers-23.3.3:
      Successfully uninstalled flatbuffers-23.3.3
  Attempting uninstall: tensorflow
    Foun

In [ ]:
!pip install pennylane-lightning[gpu]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.8/27.8 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 KB 20.3 MB/s eta 0:00:00


In [ ]:
!pip install pennylane-lightning[qubits]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Import Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from keras.models import Model
from tensorflow.keras import layers

import numpy as np
import numpy
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import re

import pennylane as qml
from pennylane import numpy as p_np

from pennylane.templates.state_preparations import MottonenStatePreparation
from pennylane.templates.layers import StronglyEntanglingLayers
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Data Prepare

In [ ]:
!git clone https://github.com/mushrafi88/solar_irradiation_forecasting

Cloning into 'solar_irradiation_forecasting'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 19 (delta 5), reused 11 (delta 2), pack-reused 0
Unpacking objects: 100% (19/19), 4.53 MiB | 2.95 MiB/s, done.


In [ ]:
df = pd.read_csv('/content/solar_irradiation_forecasting/raw_data/solar_radiation_hourly_rangamati_10_years.csv')

In [ ]:
df = df.dropna(how='any')
df.set_index('Time')
df.Time = df.Time.astype('int')
df = df.drop(index=df[(df == -999).any(axis=1)].index)
df=df.drop(columns=['CLRSKY_SFC_SW_DWN','ALLSKY_SFC_LW_DWN'])
target = 'ALLSKY_SFC_SW_DWN'
features = list(df.columns.difference([target]))

In [ ]:
df = df.reset_index(drop=True)

size = int(len(df) * 0.7)

df_train = df.loc[:size].copy()
df_test = df.loc[size:].copy()

df_train


,Time,ALLSKY_SFC_SW_DWN,ALLSKY_KT,ALLSKY_SFC_UV_INDEX,T2M,T2MDEW,TS,RH2M,PS,WS50M,SZA,PRECTOTCORR,WS10M,QV2M
0,2010010106,10.45,0.36,0.00,12.01,12.41,12.46,100.00,101.04,4.05,88.82,0.0,2.75,8.91
1,2010010107,139.84,0.50,0.20,13.69,12.87,15.45,94.69,101.15,3.92,78.62,0.0,3.04,9.16
2,2010010108,334.69,0.62,1.20,15.49,13.36,18.74,87.00,101.23,3.58,67.30,0.0,3.17,9.46
3,2010010109,514.27,0.68,2.85,18.38,12.94,21.64,70.50,101.28,4.59,57.47,0.0,4.01,9.22
4,2010010110,685.15,0.76,4.73,20.09,13.10,23.79,64.00,101.25,4.70,50.00,0.0,4.18,9.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31988,2016123013,598.53,0.69,4.65,24.44,18.01,27.44,67.25,101.08,2.40,51.57,0.0,2.25,12.76
31989,2016123014,428.91,0.60,2.75,24.29,17.89,26.30,67.38,101.02,2.33,59.67,0.0,2.20,12.70
31990,2016123015,269.48,0.55,1.02,23.82,17.76,24.51,68.75,101.00,2.24,69.90,0.0,2.05,12.63
31991,2016123016,96.29,0.45,0.10,22.76,18.36,22.15,76.12,101.02,2.45,81.19,0.0,1.84,13.06


In [ ]:
target_mean = df_train[target].mean()
target_stdev = df_train[target].std()

for c in df_train.columns:
    mean = df_train[c].mean()
    stdev = df_train[c].std()

    df_train[c] = (df_train[c] - mean) / stdev
    df_test[c] = (df_test[c] - mean) / stdev


In [ ]:
df_train = df_train.sample(500)
df_test = df_test.sample(100)

In [ ]:
X_train = df_train[features].values
y_train = df_train[target].values
X_test = df_test[features].values
y_test = df_test[target].values

# Preparation of Quantum5

In [ ]:
n_qubits = 13
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(inputs, weights):
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

n_layers = 8
weight_shapes = {"weights": (n_layers, n_qubits)}

In [ ]:
clayer_1 = tf.keras.layers.Dense(13, activation="relu")

# construct the model
inputs = tf.keras.Input(shape=(13,))
x = clayer_1(inputs)
x = tf.keras.layers.Dropout(0.1)(x)

x = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=1)(x)

model = tf.keras.Model(inputs=inputs, outputs=x)
opt = tf.keras.optimizers.SGD(learning_rate=0.2)
model.compile(opt, loss="mse", metrics=["accuracy"])

# Model Fitting

In [ ]:
model_fit = model.fit(X_train, y_train, epochs=2, batch_size=64,shuffle=True, validation_data=(X_test,y_test), verbose=2,callbacks=[keras.callbacks.ModelCheckpoint("/content/model/model_{epoch}.h5")])

Epoch 1/2
8/8 - 546s - loss: 1.0519 - accuracy: 0.0000e+00 - val_loss: 1.0746 - val_accuracy: 0.0000e+00 - 546s/epoch - 68s/step
Epoch 2/2


**Model Evaluate for Quantum**

In [ ]:
model.evaluate(X_test,y_test)

1/1 [==============================] - 1s 782ms/step - loss: 0.2480 - accuracy: 0.8182


[0.24798709154129028, 0.8181818127632141]

## Model Summary and Chart for Quantum


**Model Summary for Quantum**

In [ ]:
model.summary()

Model: "model_48"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_49 (InputLayer)       [(None, 1)]               0         
                                                                 
 dense_50 (Dense)            (None, 1)                 2         
                                                                 
 dropout_48 (Dropout)        (None, 1)                 0         
                                                                 
 keras_layer_52 (KerasLayer)  (None, 1)                16        
                                                                 
Total params: 18
Trainable params: 18
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from sklearn.ensemble import RandomForestClassifier

ml = RandomForestClassifier()
ml.fit(X_train, y_train)
ml.score(X_test,y_test)

0.6363636363636364